In [2]:
import pandas as pd

In [3]:
pkmn_sales = pd.read_csv('../data/vgchartz_pokemon_sales.csv')
pkmn_rvw = pd.read_csv('../data/pokemon_reviews.csv')
pkmn_movies = pd.read_csv('../data/pokemon_movies.csv')
pkmn_anime = pd.read_csv('../data/pokemon_animes.csv')

In [4]:
# check for dupes
pkmn_sales[pkmn_sales['title'].duplicated(keep=False)]
pkmn_rvw[pkmn_rvw['name'].duplicated(keep=False)]

#drop G/S & crystal 3ds virtual rereleases with incomplete data
pkmn_sales = pkmn_sales[pkmn_sales['game_id'] != 220575].reset_index(drop=True)
pkmn_sales = pkmn_sales[pkmn_sales['game_id'] != 205360].reset_index(drop=True)

In [5]:
#grouping columns 
core_game_info = ['title', 'console', 'publisher', 'release_year', 'game_url', 'game_id']

date_info = ['release_date', 'last_update']

sales_data = ['japan_sales', 'na_sales', 'europe_sales', 'others_sales', 'total_units', 'total_shipped']

review_data = ['meta_score', 'user_review']

In [6]:
# clean sales data columns
for col in sales_data:
    pkmn_sales[col] = (pkmn_sales[col]
        .astype(str)                      # start all as strings
        .str.replace('m', '', case=False) #remove m from unit values
        .str.strip())
    pkmn_sales[col] = pd.to_numeric(pkmn_sales[col], errors='coerce')  # convert to number

In [7]:
# remove suffixes (st, nd, rd, th) for conversion to date/time
for col in date_info:
    pkmn_sales[col] = (pkmn_sales[col]
                       .astype(str)
                       .str.replace(r'(\d+)(st|nd|rd|th)', r'\1', regex=True))
    pkmn_sales[col] = pd.to_datetime(pkmn_sales[col], format='%d %b %y', errors='coerce')

In [8]:
#create release year column
pkmn_sales['release_year'] = pkmn_sales['release_date'].dt.year

In [9]:
# normalize titles
pkmn_sales['short_title'] = (pkmn_sales['title']
    .str.replace('Pokémon', '', case=False)
    .str.replace('Pokemon', '', case=False)
    .str.replace('and', '', case=False)                             
    .str.replace('Version', '', case=False)
    .str.replace('Read the review', '', case=False) # artifact from vgchartz scraping  
    .str.replace(r'[^\w\s]', '', regex=True)  # remove punctuation                              
    .str.replace(r'\s+', ' ', regex=True)     # collapse spaces                           
    .str.strip())

pkmn_rvw['short_title'] = (pkmn_rvw['name']
    .str.replace('Pokémon', '', case=False)
    .str.replace('Pokemon', '', case=False)
    .str.replace('Version', '', case=False)
    .str.replace('and', '', case=False)
    .str.replace(':', '', case=False)
    .str.replace(r'[^\w\s]', '', regex=True)  # remove punctuation                           
    .str.replace(r'\s+', ' ', regex=True)     # collapse spaces                           
    .str.strip()
    .replace({
        'Red': 'Red Green Blue', 'Green': 'Red Green Blue', 'Blue': 'Red Green Blue',
        'Gold': 'Gold Silver', 'Silver': 'Gold Silver',
        'Ruby': 'Ruby Sapphire', 'Sapphire': 'Ruby Sapphire',
        'Diamond': 'Diamond Pearl', 'Pearl': 'Diamond Pearl',
        'Black': 'Black White', 'White': 'Black White',
        'X': 'X Y', 'Y': 'X Y',
        'Sun': 'Sun Moon', 'Moon': 'Sun Moon',
        'Ultra Sun': 'Ultra Sun Ultra Moon', 'Ultra Moon': 'Ultra Sun Ultra Moon',
        'Sword': 'Sword Shield', 'Shield': 'Sword Shield',
        'Scarlet': 'Scarlet Violet', 'Violet': 'Scarlet Violet',
        'Yellow': 'Yellow',
        'Crystal': 'Crystal',
        'Emerald': 'Emerald',
        'Platinum': 'Platinum',
        'Black 2': 'Black 2 White 2', 'White 2': 'Black 2 White 2',
        "Lets Go Pikachu": "Lets Go Pikachu Lets Go Eevee",
        "Lets Go Eevee": "Lets Go Pikachu Lets Go Eevee",
        'Brilliant Diamond': 'Brilliant Diamond Shining Pearl',
        'Shining Pearl': 'Brilliant Diamond Shining Pearl',
        'HeartGold': 'HeartGold SoulSilver', 'SoulSilver': 'HeartGold SoulSilver',
        'Omega Ruby': 'Omega Ruby Alpha Sapphire', 'Alpha Sapphire': 'Omega Ruby Alpha Sapphire',
        'FireRed': 'FireRed LeafGreen', 'LeafGreen': 'FireRed LeafGreen',
        'Legends Arceus': 'Legends Arceus', 'Mystery Dungeon Explorers of Time': 'Mystery Dungeon Explorers of Time Mystery Dungeon Explorers of Darkness', 'Mystery Dungeon Explorers of Darkness': 'Mystery Dungeon Explorers of Time Mystery Dungeon Explorers of Darkness'})) #see markdown below

Reviews on metacritic are for each individual game, but the sales data is grouped by the pairs. So, I group the games using the list by replacing each of the individual games with the new combined title column and then average the respective review colums together for the matching pair of games.   

In [10]:
pkmn_rvw_grouped = (pkmn_rvw.groupby('short_title')[['meta_score', 'user_review']].mean().reset_index())

In [11]:
# remove trailing whitespace
pkmn_sales['short_title'] = pkmn_sales['short_title'].str.strip()
pkmn_rvw_grouped['short_title'] = pkmn_rvw_grouped['short_title'].str.strip()

In [12]:
# categorization key
core_rpg = {'R/G/B', 'G/S', 'R/S', 'D/P', 'B/W', 'X/Y', 'S/M', 'Sw/Sh', 'S/V'}
core_upgrade = {'Y', 'C', 'E', 'Pt', 'B2/W2', 'US/UM'}
remake = {'LGPE', 'BD/SP', 'HG/SS', 'OR/AS', 'FR/LG'}

In [13]:
abbrev_title = {
    'Red Green Blue': 'R/G/B',
    'Gold Silver': 'G/S',
    'Ruby Sapphire': 'R/S',
    'Diamond Pearl': 'D/P',
    'Black White': 'B/W',
    'X Y': 'X/Y',
    'Sun Moon': 'S/M',
    'Ultra Sun Ultra Moon': 'US/UM',
    'Sword Shield': 'Sw/Sh',
    'Brilliant Diamond Shining Pearl': 'BD/SP',
    'Legends Arceus': 'PLA',
    'Scarlet Violet': 'S/V',
    "Lets Go Pikachu Lets Go Eevee": 'LGPE',
    'Yellow Special Pikachu Edition': 'Y',
    'Crystal': 'C',
    'Emerald': 'E',
    'Platinum': 'Pt',
    'HeartGold SoulSilver': 'HG/SS',
    'Black 2 White 2': 'B2/W2',
    'Omega Ruby Alpha Sapphire': 'OR/AS',
    'FireRed LeafGreen': 'FR/LG',
    'Mystery Dungeon Explorers of Time Mystery Dungeon Explorers of Darkness': 'MD EoT/EoD'
    }

In [14]:
# function to abbreviate titles 
def abbreviate_title(short_title):
    for key, val in abbrev_title.items():
        if key in short_title:
            return val
    return short_title

In [15]:
# apply functions
pkmn_rvw_grouped['abbr_title'] = pkmn_rvw_grouped['short_title'].apply(abbreviate_title)

pkmn_sales['abbr_title'] = pkmn_sales['short_title'].apply(abbreviate_title)

In [16]:
# function to categorize titles 
def categorize_games(abbr_title):
    if abbr_title in core_rpg:
        return "CORE_RPG"
    elif abbr_title in core_upgrade:
        return "CORE_UPGRADE"
    elif abbr_title in remake:
        return "REMAKE"
    else:
        return "SPIN_OFF"

In [17]:
# apply categorization
pkmn_sales['category'] = pkmn_sales['abbr_title'].apply(categorize_games)

In [18]:
pkmn_games = pkmn_sales.merge(pkmn_rvw_grouped, on='abbr_title', how='outer')

In [19]:
# update core_game_info to include new columns
core_game_info = ['title', 'category', 'console', 'publisher', 'release_year', 'abbr_title', 'game_url', 'game_id']

In [20]:
# reorder the columns
cols = core_game_info + date_info + sales_data + review_data
pokemon_games = pkmn_games[[col for col in cols if col in pkmn_games.columns]]

#drop obselete short_title columns
pokemon_games = pokemon_games.drop(columns=[col for col in ['short_title_x', 'short_title_y'] if col in pokemon_games.columns])

pd.set_option('display.max_colwidth', None)

#Manually verified and discovered that the NaN rows are coming from freeware titles with reviews but no sales data
#find and drop freeware titles from dataset
# print(pokemon_games[pokemon_games['game_id'].isna()])
pokemon_games = pokemon_games.dropna(subset=['total_units'])
pokemon_games

,title,category,console,publisher,release_year,abbr_title,game_url,game_id,release_date,last_update,japan_sales,na_sales,europe_sales,others_sales,total_units,total_shipped,meta_score,user_review
0,Pokémon Art Academy,SPIN_OFF,3DS,Nintendo,2014.0,Art Academy,https://www.vgchartz.com/game/81812/pokemon-art-academy/?region=All,81812.0,2014-10-24,2018-08-03,0.17,0.18,0.12,0.03,0.50,NaN,76.0,7.70
1,Pokémon Black / White Version,CORE_RPG,DS,Nintendo,2011.0,B/W,https://www.vgchartz.com/game/44573/pokemon-black-white-version/?region=All,44573.0,2011-03-06,NaT,5.66,5.51,3.20,0.80,15.18,15.64,87.0,7.70
2,Pokémon Black 2 and White 2,CORE_UPGRADE,DS,Nintendo,2012.0,B2/W2,https://www.vgchartz.com/game/70468/pokemon-black-2-and-white-2/?region=All,70468.0,2012-10-07,2018-02-25,3.16,2.79,1.79,0.43,8.16,8.52,80.0,7.90
3,Pokémon Brilliant Diamond / Shining Pearl,REMAKE,NS,Nintendo,2021.0,BD/SP,https://www.vgchartz.com/game/228356/pokemon-brilliant-diamond-shining-pearl/?region=All,228356.0,2021-11-19,2021-02-26,NaN,NaN,NaN,NaN,15.06,15.06,73.0,4.90
4,Pokémon Battle Revolution,SPIN_OFF,Wii,Nintendo,2007.0,Battle Revolution,https://www.vgchartz.com/game/4019/pokemon-battle-revolution/?region=All,4019.0,2007-06-25,NaT,0.30,0.78,0.37,0.13,1.59,1.95,53.0,6.70
6,Pokémon Beach Blank-out Blastoise / Go West Young Meowth Game Boy Advance Video,SPIN_OFF,GBA,Nintendo,2004.0,Beach Blankout Blastoise Go West Young Meowth Game Boy Advance Video,https://www.vgchartz.com/game/28110/pokemon-beach-blank-out-blastoise-go-west-young-meowth-game-boy-advance-video/?region=All,28110.0,2004-09-27,NaT,0.00,0.12,0.05,0.00,0.17,NaN,NaN,NaN
7,Pokémon Crystal Version,CORE_UPGRADE,GBC,Nintendo,2001.0,C,https://www.vgchartz.com/game/4021/pokemon-crystal-version/?region=All,4021.0,2001-07-29,2018-01-06,1.29,2.55,1.56,0.99,6.39,6.39,NaN,NaN
9,Pokémon Channel,SPIN_OFF,GC,Nintendo,2003.0,Channel,https://www.vgchartz.com/game/1760/pokemon-channel/?region=All,1760.0,2003-12-01,NaT,0.07,0.24,0.06,0.01,0.38,NaN,55.0,6.40
10,Pokémon Colosseum,SPIN_OFF,GC,Nintendo,2004.0,Colosseum,https://www.vgchartz.com/game/1761/pokemon-colosseum/?region=All,1761.0,2004-03-22,NaT,0.70,1.21,0.57,0.07,2.54,2.41,73.0,8.00
11,Pokémon Conquest,SPIN_OFF,DS,Nintendo,2012.0,Conquest,https://www.vgchartz.com/game/70562/pokemon-conquest/?region=All,70562.0,2012-06-18,2018-07-30,0.37,0.56,0.05,0.05,1.02,NaN,80.0,8.30


In [21]:
pokemon_games['release_date'] = pd.to_datetime(pokemon_games['release_date'])
pkmn_movies['release_date_na'] = pd.to_datetime(pkmn_movies['release_date_na'])

In [22]:
from datetime import timedelta

# Convert dates (ensure consistent variable names)
pokemon_games['release_date'] = pd.to_datetime(pokemon_games['release_date'])
pkmn_movies['release_date_na'] = pd.to_datetime(pkmn_movies['release_date_na'])

# Optimized movie flag function
def has_nearby_movie(game_date):
    """Check if any movie released within ±180 days (~6 mo.) of game release"""
    time_window = timedelta(days=180)
    return any((abs(pkmn_movies['release_date_na'] - game_date) <= time_window))

# Apply to games dataframe
pokemon_games['near_movie_release'] = (
    pokemon_games['release_date']
    .apply(has_nearby_movie)
    .astype(int))

In [23]:
pokemon_games

,title,category,console,publisher,release_year,abbr_title,game_url,game_id,release_date,last_update,japan_sales,na_sales,europe_sales,others_sales,total_units,total_shipped,meta_score,user_review,near_movie_release
0,Pokémon Art Academy,SPIN_OFF,3DS,Nintendo,2014.0,Art Academy,https://www.vgchartz.com/game/81812/pokemon-art-academy/?region=All,81812.0,2014-10-24,2018-08-03,0.17,0.18,0.12,0.03,0.50,NaN,76.0,7.70,1
1,Pokémon Black / White Version,CORE_RPG,DS,Nintendo,2011.0,B/W,https://www.vgchartz.com/game/44573/pokemon-black-white-version/?region=All,44573.0,2011-03-06,NaT,5.66,5.51,3.20,0.80,15.18,15.64,87.0,7.70,1
2,Pokémon Black 2 and White 2,CORE_UPGRADE,DS,Nintendo,2012.0,B2/W2,https://www.vgchartz.com/game/70468/pokemon-black-2-and-white-2/?region=All,70468.0,2012-10-07,2018-02-25,3.16,2.79,1.79,0.43,8.16,8.52,80.0,7.90,1
3,Pokémon Brilliant Diamond / Shining Pearl,REMAKE,NS,Nintendo,2021.0,BD/SP,https://www.vgchartz.com/game/228356/pokemon-brilliant-diamond-shining-pearl/?region=All,228356.0,2021-11-19,2021-02-26,NaN,NaN,NaN,NaN,15.06,15.06,73.0,4.90,1
4,Pokémon Battle Revolution,SPIN_OFF,Wii,Nintendo,2007.0,Battle Revolution,https://www.vgchartz.com/game/4019/pokemon-battle-revolution/?region=All,4019.0,2007-06-25,NaT,0.30,0.78,0.37,0.13,1.59,1.95,53.0,6.70,1
6,Pokémon Beach Blank-out Blastoise / Go West Young Meowth Game Boy Advance Video,SPIN_OFF,GBA,Nintendo,2004.0,Beach Blankout Blastoise Go West Young Meowth Game Boy Advance Video,https://www.vgchartz.com/game/28110/pokemon-beach-blank-out-blastoise-go-west-young-meowth-game-boy-advance-video/?region=All,28110.0,2004-09-27,NaT,0.00,0.12,0.05,0.00,0.17,NaN,NaN,NaN,1
7,Pokémon Crystal Version,CORE_UPGRADE,GBC,Nintendo,2001.0,C,https://www.vgchartz.com/game/4021/pokemon-crystal-version/?region=All,4021.0,2001-07-29,2018-01-06,1.29,2.55,1.56,0.99,6.39,6.39,NaN,NaN,1
9,Pokémon Channel,SPIN_OFF,GC,Nintendo,2003.0,Channel,https://www.vgchartz.com/game/1760/pokemon-channel/?region=All,1760.0,2003-12-01,NaT,0.07,0.24,0.06,0.01,0.38,NaN,55.0,6.40,0
10,Pokémon Colosseum,SPIN_OFF,GC,Nintendo,2004.0,Colosseum,https://www.vgchartz.com/game/1761/pokemon-colosseum/?region=All,1761.0,2004-03-22,NaT,0.70,1.21,0.57,0.07,2.54,2.41,73.0,8.00,1
11,Pokémon Conquest,SPIN_OFF,DS,Nintendo,2012.0,Conquest,https://www.vgchartz.com/game/70562/pokemon-conquest/?region=All,70562.0,2012-06-18,2018-07-30,0.37,0.56,0.05,0.05,1.02,NaN,80.0,8.30,1


In [24]:
pokemon_games.to_csv('pokemon_games.csv', index=False)

In [25]:
###############################################################################

In [26]:
#Clean and prep Pokemon Movies data set
pkmn_movies.dtypes

order                       int64
english title              object
japanese_title             object
release_date_jp            object
release_date_na    datetime64[ns]
release_year_na             int64
release_year_jp             int64
dtype: object

In [27]:
# # convert to datetime if they're not already
# pkmn_movies['North American release date'] = pd.to_datetime(pkmn_movies['North American release date'])
# pkmn_movies['Japanese release date'] = pd.to_datetime(pkmn_movies['Japanese release date'])

# # extract the year 
# pkmn_movies['release_year_na'] = pkmn_movies['North American release date'].dt.year
# pkmn_movies['release_year_jp'] = pkmn_movies['Japanese release date'].dt.year

In [28]:
# pkmn_movies.to_csv('pokemon_movies.csv', index=False)

In [29]:
###############################################################################

In [32]:
pkmn_anime.dtypes

Season             int64
Season name       object
Episodes           int64
First released    object
Last released     object
dtype: object

In [ ]:
# convert to datetime if they're not already
pkmn_anime['North American release date'] = pd.to_datetime(pkmn_anime['North American release date'])
pkmn_anime['Japanese release date'] = pd.to_datetime(pkmn_anime['Japanese release date'])

# extract the year 
pkmn_anime['release_year_na'] = pkmn_anime['North American release date'].dt.year
pkmn_anime['release_year_jp'] = pkmn_anime['Japanese release date'].dt.year

In [ ]:
###############################################################################

In [ ]:
remake_mapping = {
    'FR/LG': 'R/G/B',
    'HG/SS': 'G/S',
    'OR/AS': 'R/S',
    'LGPE': 'R/G/B',
    'BD/SP': 'D/P'}

remake_mapping_df = pd.DataFrame(list(remake_mapping.items()), columns=['remake', 'original'])

remake_mapping_df.to_csv('remake_mapping.csv', index=False)

# Use the mapping to add the original title information to the remakes
remakes = pkmn_games[pkmn_games['abbr_title'].isin(remake_mapping.keys())].copy()
remakes['original_title'] = remakes['abbr_title'].map(remake_mapping)
remakes['type'] = 'Remake'

# Now, for the originals, we'll just take the original titles directly from the mapping
originals = pkmn_games[pkmn_games['abbr_title'].isin(remake_mapping.values())].copy()
originals['original_title'] = originals['abbr_title']
originals['type'] = 'Original'

og_remakes = pd.concat([remakes, originals], ignore_index=True)

og_remakes.to_csv('remakes_comparison.csv', index=False)